In [4]:
#importing important libaries 
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [118]:
#Creating empty lists for the information that will be stored
Year = []
Brand = []
Model = []
State = []
Milage = []
Color = []
Price = []
Body = []

In [8]:
#A list of different body types which is used in the function
body_types = ["SUV", "Sedan", "Convertible", "Wagon", "Hatchback", "Pickup", "Van", "Utility", "Cruiser"]

In [56]:
#Creating the web scraping function
def scrape_cars():
    cars = soup.find_all("div", class_ = "sv-container")
    for car in cars:    
        car_name = car.find("div", class_ = "sv-title-container").span.a.text
        div1 = car.find("div", class_ = "sv-desc-container")
        location = div1.find_all("span", class_ = "value")[2].text.strip()
        div2 = car.find("div", class_ = "sv-desc-container middle")
        milage = div2.find_all("span", class_ = "value")[0].text
        color = div2.find_all("span", class_ = "value")[1].text
        year = car_name.split(" ")[0]
        brand = car_name.split(" ")[1]
        model = car_name.split(" ")[2]
        state = location.split(",")[0]
        div3 = car.find("div", class_ = "price-pane")
        status = div3.find("span", class_ = "label")
        if status is not None:
            status = status.text
            if "buy" in status:
                price_1 = div3.find("span", class_ = "value").text.replace(",", "")
                price = int(price_1.split(" ")[1])
                Year.append(year)
                Brand.append(brand)
                Model.append(model)
                State.append(state)
                Milage.append(milage)
                Color.append(color)
                Price.append(price)
                match_found = False
                for types in body_types:
                    if types.lower() in car_name.lower():
                        Body.append(types)
                        match_found = True
                        break
                if not match_found:
                    Body.append("Unknown")

In [ ]:
#Calling the web scraping function for the first 850 pages
for i in range(1, 851):
    url = "https://www.auctionexport.com/en/cars/clean/page/"
    page = requests.get(url + str(i))
    soup = BeautifulSoup(page.text, "html")
    scrape_cars()

In [122]:
#Turning the lists into a pandas dataframe
data = {
    "Year": Year,
    "Brand": Brand,
    "Model": Model,
    "State": State,
    "Milage": Milage,
    "Color": Color,
    "Price": Price,
    "Body": Body
}
df = pd.DataFrame(data)
df

,Year,Brand,Model,State,Milage,Color,Price,Body
0,2007,BMW,328XI,Massachusetts,153962,GRAY,550,Unknown
1,2002,BMW,X5,Mississippi,236076,GREEN,725,Wagon
2,2010,HONDA,CR-V,Florida,257802,GRAY,1325,Wagon
3,2001,HONDA,CIVIC,Florida,1,RED,675,Unknown
4,2009,HUMMER,H3,Florida,218756,BLACK,2400,SUV
...,...,...,...,...,...,...,...,...
1082,2010,FORD,EDGE,Pennsylvania,166606,WHITE,1000,Wagon
1083,2004,HYUNDAI,ACCENT,New Jersey,254000,GREEN,325,Hatchback
1084,2021,CHEVROLET,EQUINOX,Illinois,91671,WHITE,6000,Utility
1085,2009,FORD,EDGE,Pennsylvania,220308,BURGUNDY,825,Wagon


In [76]:
#Exporting the table into a csv file
df.to_csv("Auction_Export")